In [1]:
import xml.etree.ElementTree as ETree
import pandas as pd

  
# give the path where you saved the xml file
# inside the quotes
xmldata = "NEWS2018_M-EnHi_trn.xml"
prstree = ETree.parse(xmldata)
root = prstree.getroot()
  
print(root)
items = []
all_items = []
  
for storeno in root.iter('Name'):
    
    ID = storeno.attrib.get('ID')
    SourceName = storeno.find('SourceName').text
    TargetName = storeno.find('TargetName').text
  
    items = [ID, SourceName, TargetName]
    all_items.append(items)
xmldata = "NEWS2018_M-EnHi_dev.xml"
prstree = ETree.parse(xmldata)
root = prstree.getroot()
  
print(root)
  
for storeno in root.iter('Name'):
    
    ID = storeno.attrib.get('ID')
    SourceName = storeno.find('SourceName').text
    TargetName = storeno.find('TargetName').text
  
    items = [ID, SourceName, TargetName]
    all_items.append(items)
  
lines = pd.DataFrame(all_items, columns=[
  'source', 'english', 'hindi'])

<Element 'TransliterationCorpus' at 0x00000214E64A9598>
<Element 'TransliterationCorpus' at 0x00000214A9978A48>


In [2]:
len(lines)

13937

In [3]:
import nltk
from collections import Counter
from tqdm import tqdm_notebook
import numpy as np
import tensorflow as tf
from tensorflow.contrib import seq2seq
from tensorflow.contrib.rnn import DropoutWrapper
import random

In [4]:
MAX_SEQ_LEN = 20
BATCH_SIZE = 64
#number of samples
N = len(lines)

In [5]:
class Lang:
    def __init__(self, counter, vocab_size):
        self.word2id = {}
        self.id2word = {}
        self.pad = "<PAD>"
        self.sos = "<SOS>"
        self.eos = "<EOS>"
        self.unk = "<UNK>"
        
        self.ipad = 0
        self.isos = 1
        self.ieos = 2
        self.iunk = 3
        
        self.word2id[self.pad] = 0
        self.word2id[self.sos] = 1
        self.word2id[self.eos] = 2
        self.word2id[self.unk] = 3
        
        self.id2word[0] = self.pad
        self.id2word[1] = self.sos
        self.id2word[2] = self.eos
        self.id2word[3] = self.unk
        
        curr_id = 4
        for w, c in counter.most_common(vocab_size):
            self.word2id[w] = curr_id
            self.id2word[curr_id] = w
            curr_id += 1
            
    def encodeSentence(self, s, max_len=-1):
        wseq = s.lower().strip()
        if max_len == -1:
            return [self.word2id[w] if w in self.word2id else self.iunk for w in wseq]
        else:
            return ([self.word2id[w] if w in self.word2id else self.iunk for w in wseq] + [self.ieos] + [self.ipad]*max_len)[:max_len]
        
    def encodeSentence2(self, s, max_len=-1):
        wseq = wseq = s.lower().strip()
        return min(max_len, len(wseq)+1), \
            ([self.word2id[w] if w in self.word2id else self.iunk for w in wseq] + \
                [self.ieos] + [self.ipad]*max_len)[:max_len]
    
    def decodeSentence(self, id_seq):
        id_seq = np.array(id_seq + [self.ieos])
        j = np.argmax(id_seq==self.ieos)
        s = ''.join([self.id2word[x] for x in id_seq[:j]])
        s = s.replace(self.unk, "UNK")
        return s

In [6]:
hi_counter = Counter()
hi_sentences=[]
en_counter = Counter()
en_sentences=[]


hi_sentences=lines['hindi'][:N]
en_sentences=lines['english'][:N]
for line in tqdm_notebook(hi_sentences):
    for w in line.strip():
        hi_counter[w] += 1
for line in tqdm_notebook(en_sentences):
    for w in line.strip():
        en_counter[w] += 1

c:\users\nain\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:9: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  if __name__ == '__main__':


  0%|          | 0/13937 [00:00<?, ?it/s]

c:\users\nain\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:12: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  if sys.path[0] == '':


  0%|          | 0/13937 [00:00<?, ?it/s]

In [7]:
# A few sample hindi characters
print("Most common hi characters in dataset:\n", hi_counter.most_common(5))

print("\nTotal (hi)characters gathered from dataset:",len(hi_counter))

# A few sample english characters
print("\nMost common en characters in dataset:\n", en_counter.most_common(5))

print("\nTotal (en)characters gathered from dataset:", len(en_counter))

Most common hi characters in dataset:
 [('ा', 9549), ('र', 9487), ('्', 8129), (' ', 6599), ('न', 5857)]

Total (hi)characters gathered from dataset: 83

Most common en characters in dataset:
 [('a', 23354), ('r', 9809), ('i', 9316), ('n', 8959), ('e', 8951)]

Total (en)characters gathered from dataset: 44


In [8]:
en_lang = Lang(en_counter, len(en_counter))
hi_lang = Lang(hi_counter, len(hi_counter))

In [9]:
print("Test en encoding:", en_lang.encodeSentence("Shukriya"))

print("Test en decoding:", en_lang.decodeSentence(en_lang.encodeSentence("Shukriya", 10)))

print("Test hindi encoding:", hi_lang.encodeSentence("शुक्रिया", 10))

print("Test hindi decoding:", hi_lang.decodeSentence((hi_lang.encodeSentence("शुक्रिया", 10))))

Test en encoding: [11, 9, 17, 18, 5, 6, 23, 4]
Test en decoding: shukriya
Test hindi encoding: [30, 23, 14, 6, 5, 9, 20, 4, 2, 0]
Test hindi decoding: शुक्रिया


In [10]:
VE = len(en_lang.word2id)
VH = len(hi_lang.word2id)

In [11]:
en_word_emb_matrix = tf.get_variable("en_word_emb_matrix", (VE, 300), dtype=tf.float32)
hi_word_emb_matrix = tf.get_variable("hi_word_emb_matrix", (VH, 300), dtype=tf.float32)

In [12]:
keep_prob = tf.placeholder(tf.float32)

input_ids = tf.placeholder(tf.int32, (None, MAX_SEQ_LEN))
input_lens = tf.placeholder(tf.int32, (None, ))

ph_target_ids = tf.placeholder(tf.int32, (None, MAX_SEQ_LEN))
target_lens = tf.placeholder(tf.int32, (None, ))

In [13]:
# Add SOS or GO symbol
target_ids = tf.concat([tf.fill([BATCH_SIZE,1], hi_lang.isos), ph_target_ids], -1)

In [14]:
input_emb = tf.nn.embedding_lookup(en_word_emb_matrix, input_ids)
target_emb = tf.nn.embedding_lookup(hi_word_emb_matrix, target_ids[:, :-1])

In [15]:
input_emb.shape

TensorShape([Dimension(None), Dimension(20), Dimension(300)])

In [16]:
encoder_cell = tf.nn.rnn_cell.GRUCell(128) # 128 is the dimension of hidden state
encoder_cell = DropoutWrapper(encoder_cell, output_keep_prob=keep_prob) # Adding Dropout for regularization

Instructions for updating:
This class is equivalent as tf.keras.layers.GRUCell, and will be replaced by that in Tensorflow 2.0.


In [17]:
enc_outputs, enc_state = tf.nn.dynamic_rnn(
    encoder_cell, # The encoder GRU cell
    input_emb, # Embedded input sequence
    sequence_length=input_lens, # Sequence lengths of individual inputs in a batch
    initial_state=encoder_cell.zero_state(BATCH_SIZE, dtype=tf.float32)
)

Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [18]:
# Confirm the shape of the final hidden state
enc_state.shape

TensorShape([Dimension(64), Dimension(128)])

In [19]:
decoder_cell = tf.nn.rnn_cell.GRUCell(128)
decoder_cell = DropoutWrapper(decoder_cell, output_keep_prob=keep_prob)

In [20]:
output_projection = tf.layers.Dense(len(hi_lang.word2id))

In [21]:
helper = seq2seq.TrainingHelper(target_emb, target_lens)
decoder = seq2seq.BasicDecoder(decoder_cell, helper, enc_state, output_projection)
outputs, _, outputs_lens = seq2seq.dynamic_decode(decoder, maximum_iterations=MAX_SEQ_LEN, 
                                                  impute_finished=False, swap_memory=True)
output_max_len = tf.reduce_max(outputs_lens)

In [22]:
# Using the decoder_cell without dropout here.
infer_helper = seq2seq.GreedyEmbeddingHelper(hi_word_emb_matrix, tf.fill([BATCH_SIZE, ], hi_lang.isos), hi_lang.ieos)
infer_decoder = seq2seq.BasicDecoder(decoder_cell, infer_helper, enc_state, output_projection)
infer_output = seq2seq.dynamic_decode(infer_decoder, maximum_iterations=MAX_SEQ_LEN, swap_memory=True)

In [23]:
# Sequence mask:
# To make sure we don't back-propagate error from output of length positions
masks = tf.sequence_mask(target_lens, output_max_len, dtype=tf.float32, name='masks')

# Loss function - weighted softmax cross entropy
cost = seq2seq.sequence_loss(
    outputs[0],
    target_ids[:, 1:(output_max_len + 1)],
    masks)

# Optimizer
optimizer = tf.train.AdamOptimizer(0.0001)

In [24]:
train_op = optimizer.minimize(cost)

In [25]:
init = tf.global_variables_initializer()

In [26]:
sess_config = tf.ConfigProto()
sess_config.gpu_options.allow_growth = True

In [27]:
sess = tf.InteractiveSession(config=sess_config)
sess.run(init)

In [28]:
random.seed(41)

In [29]:
parallel = list(zip(en_sentences, hi_sentences))

In [30]:
en_sentences

0              aabhaa
1             aabheer
2           aabhijaat
3               aabid
4             aabshar
             ...     
13932    yuvraj singh
13933    zahoor elahi
13934          zaleel
13935            zayb
13936          zorion
Name: english, Length: 13937, dtype: object

In [31]:
random.shuffle(parallel)

In [32]:
parallel

[('kara-kum', 'काराकुम'),
 ('daler', 'दलेर'),
 ('sulwyn', 'सुल्विन'),
 ('manning', 'मैनिंग'),
 ('shashibala', 'शशिबाला'),
 ('talisma', 'तलीस्मा'),
 ('thirunarayan', 'तिरुनारायण'),
 ('medal of suvorov', 'मेडल ऑफ सुवोरोव'),
 ('errin', 'एरिन'),
 ('gurudwara tibbi sahib', 'गुरूद्वारा टिब्बी साहिब'),
 ('amadhya', 'अमध्य'),
 ('radhika', 'राधिका'),
 ('paul hitchcock', 'पॉल हिचकॉक'),
 ('kush', 'कुश'),
 ('aru', 'अरु'),
 ('fort douaumont', 'फोर्ट डॉमोंट'),
 ('hindu american', 'हिन्दू अमेरिकन'),
 ("saint george's", 'सेंट जॉर्ज'),
 ('abdul haakim', 'अब्दुल हाकिम'),
 ('ebra', 'एब्रा'),
 ('randheer', 'रणधीर'),
 ('roma', 'रोमा'),
 ('dartmoor', 'डार्टमूर'),
 ('moghe', 'मोघे'),
 ('qaylah', 'कयलाह'),
 ('gaon hamara shehar tumhara', 'गाँव हमारा शहर तुम्हारा'),
 ('pamela', 'पामेला'),
 ('corey collymore', 'कॉरे कॉलीमर'),
 ('shankar khan', 'शंकर खान'),
 ('wayne', 'वेन'),
 ('samsherbaz', 'समशेरबाज़'),
 ('pocono', 'पॉकॉनो'),
 ('paramanand', 'परमानन्द'),
 ('kenya', 'केन्या'),
 ('shatrughn', 'शत्रुघ्न'),
 ('sawa

In [33]:
#split data into 95% and 5% for train and validation
train_n = int(0.95*N)
valid_n = N - train_n

In [34]:
train_pairs = parallel[:train_n].copy()
valid_pairs = parallel[train_n:]

In [35]:
def small_test():
    all_bleu = []
    smoothing = nltk.translate.bleu_score.SmoothingFunction().method7
    for m in range(0, valid_n, BATCH_SIZE):
        # print(f"Status: {m}/{N}", end='\r')
        n = m + BATCH_SIZE
        if n > valid_n:
            break

        input_batch = np.zeros((BATCH_SIZE, MAX_SEQ_LEN), dtype=np.int32)
        input_lens_batch = np.zeros((BATCH_SIZE,), dtype=np.int32)
        for i in range(m, n):
            b,a = en_lang.encodeSentence2(valid_pairs[i][0], MAX_SEQ_LEN)
            input_batch[i-m,:] = a
            input_lens_batch[i-m] = b



        feed_dict={
            input_ids: input_batch,
            input_lens: input_lens_batch,

            keep_prob: 1.0
        }
        pred_batch = sess.run(infer_output[0].sample_id, feed_dict=feed_dict)
        for k, pred_ in enumerate(pred_batch):
            pred_s = hi_lang.decodeSentence(list(pred_))
            ref = valid_pairs[m+k][1]
            try:
                _bx = nltk.translate.bleu_score.sentence_bleu(
                    [ref],
                    pred_s,
                    weights=[1/4]*4,
                    smoothing_function=smoothing)
            except ZeroDivisionError:
                _bx = 0
            all_bleu.append(_bx)

    print(f"BLEU Score: {np.mean(all_bleu)}")

In [36]:
for _e in range(200):
    # Mix things up a bit.
    random.shuffle(train_pairs)
    pbar = tqdm_notebook(range(0, train_n, BATCH_SIZE))
    batch_loss = 0
    bxi = 0
    for m in pbar:
        n = m + BATCH_SIZE
        if n <= train_n:
            # print("Epoch Complete... \n")

            input_batch = np.zeros((BATCH_SIZE, MAX_SEQ_LEN), dtype=np.int32)
            input_lens_batch = np.zeros((BATCH_SIZE,), dtype=np.int32)
            for i in range(m, n):
                b,a = en_lang.encodeSentence2(train_pairs[i][0], MAX_SEQ_LEN)
                input_batch[i-m,:] = a
                input_lens_batch[i-m] = b

            target_batch = np.zeros((BATCH_SIZE, MAX_SEQ_LEN), dtype=np.int32)
            target_lens_batch = np.zeros((BATCH_SIZE,), dtype=np.int32)
            for i in range(m, n):
                b,a = hi_lang.encodeSentence2(train_pairs[i][1], MAX_SEQ_LEN)
                target_batch[i-m,:] = a
                target_lens_batch[i-m] = b

            feed_dict={
                input_ids: input_batch,
                input_lens: input_lens_batch,
                ph_target_ids: target_batch,
                target_lens: target_lens_batch,
                keep_prob: 0.8 
            }
            sess.run(train_op, feed_dict=feed_dict)
            batch_loss += sess.run(cost, feed_dict=feed_dict)
            pbar.set_description(f"Epoch: {_e} >> Loss: {batch_loss/(bxi+1):2.2F}:")
            bxi += 1
            if (1 + n//BATCH_SIZE) % 100 == 0:
                small_test()

c:\users\nain\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


  0%|          | 0/207 [00:00<?, ?it/s]

BLEU Score: 0.021706666905198967
BLEU Score: 0.04320409049441479


  0%|          | 0/207 [00:00<?, ?it/s]

BLEU Score: 0.06043606218612607
BLEU Score: 0.0764802138606902


...

BLEU Score: 0.5098421752327245
BLEU Score: 0.5105372743413399


  0%|          | 0/207 [00:00<?, ?it/s]

BLEU Score: 0.5042403443223733
BLEU Score: 0.5022238887135737


  0%|          | 0/207 [00:00<?, ?it/s]

BLEU Score: 0.5037039269610649
BLEU Score: 0.5087270435881411


In [37]:
def transliterate(s):
    input_batch = np.zeros((BATCH_SIZE, MAX_SEQ_LEN), dtype=np.int32)
    input_lens_batch = np.zeros((BATCH_SIZE,), dtype=np.int32)
    b,a = en_lang.encodeSentence2(s, MAX_SEQ_LEN)
    input_batch[0, :] = a
    input_lens_batch[0] = b
    
    feed_dict={
        input_ids: input_batch,
        input_lens: input_lens_batch,
        #target_ids: target_batch,
        #target_lens: target_lens_batch,
        keep_prob: 1.0
    }
    pred_batch = sess.run(infer_output[0].sample_id, feed_dict=feed_dict)
    pred_ = pred_batch[0]
    pred_s = hi_lang.decodeSentence(list(pred_))
    # ref = valid_pairs[m+k][1]
    return pred_s

some examples:-

In [38]:
transliterate("saya")

'साया'

In [47]:
transliterate('shubham')

'शुभाम'

In [40]:
transliterate('ayushi')

'आयुषी'

In [41]:

#testing XML file
xmldata = "NEWS2018_M-EnHi_tst.xml"
prstree = ETree.parse(xmldata)
root = prstree.getroot()
  
print(root)
store_items = []
all_items = []
  
for storeno in root.iter('Name'):
    
    store_Nr = storeno.attrib.get('ID')
    itemsF = storeno.find('SourceName').text
   
  
    store_items = [store_Nr, itemsF]
    all_items.append(store_items)

  
lines2 = pd.DataFrame(all_items, columns=[
  'source', 'english'])

<Element 'TransliterationCorpus' at 0x00000214C9EF2868>


In [48]:
import random
 
#testing on test XML file 
#example 1
x=random.choice(lines2['english'])
print('english:-',x)
print('hindi:- ',transliterate(x))

english:- kalpataru
hindi pridiction:-  काल्पतुरा


In [49]:
#example 2
x=random.choice(lines2['english'])
print('english:-',x)
print('hindi:- ',transliterate(x))

english:- gyanlata
hindi:-  ग्यानलता


In [50]:
#example 3
x=random.choice(lines2['english'])
print('english:-',x)
print('hindi:- ',transliterate(x))

english:- vandita
hindi:-  वंदिता


In [51]:
#example 4
x=random.choice(lines2['english'])
print('english:-',x)
print('hindi:- ',transliterate(x))

english:- malvin
hindi:-  मल्विन


In [55]:
#example 5
x=random.choice(lines2['english'])
print('english:-',x)
print('hindi:- ',transliterate(x))

english:- herriott
hindi:-  हेरिटो
